In [8]:
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import datetime as dt
import time

In [ ]:
# portfolio of crypto
crypto = ['BNB-USD', 'ETH-USD', 'BTC-USD', 'DOGE-USD', 'LINK-USD', 'SOL-USD']

#### Download historical data from yf API

In [ ]:
import yfinance as yf

start_date = "2020-01-01"
end_date = datetime.now().strftime("%Y-%m-%d")

# Download historical data from yf API
data = yf.download(crypto, start=start_date, end=end_date, group_by='ticker')

# download data to csv
data.to_csv("crypto.csv")

#### Download analysis from csv downloaded data

In [ ]:
# Load the CSV with MultiIndex columns (Tickers, OHLCV)
df = pd.read_csv("crypto.csv", header=[0,1], index_col=0)

# Drop any rows that are completely NaN (e.g. 'Date' row)
df = df.dropna(how='all')

# Forward fill missing values for each column
df = df.fillna(method='ffill')

# Convert all values to float
df = df.astype(float)

# rename tickers to more readable names, ^DJI -> DJ, ^GSPC -> SP, ^IXIC -> NQ
df.columns = pd.MultiIndex.from_tuples([
    (
        col[0].replace('BNB-USD', 'BNB').replace('ETH-USD', 'ETH').replace('BTC-USD', 'BTC').replace('DOGE-USD', 'DOGE').replace('LINK-USD', 'LINK').replace('SOL-USD', 'SOL'),
        col[1]
    ) for col in df.columns
])


# Show the result
df.head()

BABA                                                  \
                  Open        High         Low       Close      Volume   
Date                                                                     
2020-01-02  205.703946  208.913905  205.646952  208.714478  15873500.0   
2020-01-03  205.466521  207.226302  205.143613  206.083817   8604500.0   
2020-01-06  204.079962  206.235774  203.320203  205.741928  11885500.0   
2020-01-07  206.691621  207.926227  205.789413  206.682129   9388000.0   
2020-01-08  205.703941  209.550194  205.438028  207.033508  11959100.0   

                  MSFT                                                  ...  \
                  Open        High         Low       Close      Volume  ...   
Date                                                                    ...   
2020-01-02  151.040826  152.895777  150.612762  152.791138  22622100.0  ...   
2020-01-03  150.603260  152.153802  150.355924  150.888626  21116200.0  ...   
2020-01-06  149.423644  151.345190  148.881420  151.278595  20813700.0  ...   
2020-01-07  151.554518  151.887449  149.652001  149.899323  21634100.0  ...   
2020-01-08  151.183539  152.962403  150.251310  152.287003  27746500.0  ...   

                NVDA                                                  AAPL  \
                Open      High       Low     Close       Volume       Open   
Date                                                                         
2020-01-02  5.942537  5.971409  5.892010  5.971409  237536000.0  71.545897   
2020-01-03  5.851688  5.919639  5.826798  5.875832  205384000.0  71.765674   
2020-01-06  5.782494  5.905701  5.756360  5.900474  262636000.0  70.954181   
2020-01-07  5.928847  6.017705  5.883796  5.971907  314856000.0  72.415330   
2020-01-08  5.967677  6.024426  5.927603  5.983109  277108000.0  71.768101   

                                                          
                 High        Low      Close       Volume  
Date                                                      
2020-01-02  72.598899  71.292311  72.538521  135480400.0  
2020-01-03  72.594063  71.608692  71.833298  146322800.0  
2020-01-06  72.444313  70.703005  72.405670  118387200.0  
2020-01-07  72.671333  71.845362  72.065140  108872000.0  
2020-01-08  73.526318  71.768101  73.224426  132079200.0  

[5 rows x 30 columns]

In [12]:
# # Here you can plot the charts for every ticker

# # For every ticker, plot Close
# import matplotlib.pyplot as plt

# for ticker in df.columns.levels[0]:
#     plt.figure(figsize=(12, 6))
#     plt.plot(df[ticker]['Close'], label=ticker)
#     plt.title(f"{ticker} Close Price")
#     plt.xlabel("Date")
#     plt.ylabel("Close Price")
#     plt.legend()
#     plt.show()

#### Plot of Volume Profile for All crypto

In [ ]:
import plotly.io as pio
pio.renderers.default = 'vscode'
import plotly.express as px
import plotly.graph_objects as go

for c in crypto:
    # Use the crypto data from our df
    df_crypto = df[c][['Close', 'Volume']]

    # Convert the crypto to datetime if needed
    if not isinstance(df_crypto.index, pd.DatetimeIndex):
        df_crypto.index = pd.to_datetime(df_crypto.index)

    # plot
    hist = go.Histogram(
        y=df_crypto['Close'],
        x=df_crypto['Volume'],
        nbinsy=150,
        orientation='h',
        name='Volume Profile',
        marker_color='#B0C4DE',
        opacity=0.5,
        xaxis='x2',
    )

    line = go.Scatter(
        x=df_crypto.index,
        y=df_crypto['Close'],
        mode='lines',
        name='Close',
        marker_color='blue',
        xaxis='x',
        yaxis='y',
    )

    fig = go.Figure([hist, line])

    fig.update_layout(
        xaxis=dict(
            title='Date',
            anchor='y',
            domain=[0, 1],
            showgrid=False,
            side='bottom'
        ),
        xaxis2=dict(
            title='Sum of Volume',
            overlaying='x',
            side='top',
            showgrid=False
        ),
        yaxis=dict(
            title='Close Price',
            showgrid=True
        ),
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1
        ),
        height=600,
        title=f'Volume Profile & {c} Close Prices'
    )

    fig.show()

In [ ]:
# !pip install -U kaleido

# kaleido library is used for static image export of Plotly figures, 
# required for saving figures as images

In [ ]:
# Same iteration for all crypto as previous cell, just added the image saving with kaleido
import plotly.io as pio
pio.renderers.default = 'vscode'
import plotly.express as px
import plotly.graph_objects as go

crypto_ = ['BNB-USD', 'ETH-USD', 'BTC-USD', 'DOGE-USD', 'LINK-USD', 'SOL-USD']

for c in crypto_:
    df_crypto = df[c][['Close', 'Volume']]

    hist = go.Histogram(
        y=df_crypto['Close'],
        x=df_crypto['Volume'],
        nbinsy=150,
        orientation='h',
        name='Volume Profile',
        marker_color='#B0C4DE',
        opacity=0.5,
        xaxis='x2',
    )

    line = go.Scatter(
        x=df_crypto.index,
        y=df_crypto['Close'],
        mode='lines',
        name='Close',
        marker_color='blue',
        xaxis='x',
        yaxis='y',
    )

    fig = go.Figure([hist, line])

    fig.update_layout(
        xaxis=dict(
            title='Date',
            anchor='y',
            domain=[0, 1],
            showgrid=False,
            side='bottom'
        ),
        xaxis2=dict(
            title='Sum of Volume',
            overlaying='x',
            side='top',
            showgrid=False
        ),
        yaxis=dict(
            title='Close Price',
            showgrid=True
        ),
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1
        ),
        height=600,
        title=f'Volume Profile & {c} Close Prices'
    )

    fig.show()

    # Salva il grafico come JPG
    fig.write_image(f"{c}_volume_profile.jpg", format="jpg")